# 🎨 使用 GitHub 模型的代理設計模式 (Python)

## 📋 學習目標

此筆記本展示了使用 Microsoft Agent Framework 與 GitHub 模型整合來構建智能代理的基本設計模式。您將學習經過驗證的模式和架構方法，使代理更加穩健、可維護且高效。

**涵蓋的核心設計模式：**
- 🏗️ **代理工廠模式**：標準化的代理創建與配置
- 🔧 **工具註冊模式**：管理代理功能的有序方法
- 🧵 **對話管理**：多輪交互的有效模式
- 🔄 **回應處理**：處理代理輸出的最佳實踐

## 🎯 關鍵架構概念

### 設計原則
- **職責分離**：代理邏輯、工具和配置之間的明確界限
- **組合性**：從可重用組件構建複雜代理
- **可擴展性**：允許輕鬆添加新功能的模式
- **可測試性**：設計便於單元測試和驗證

### GitHub 模型整合
- **API 兼容性**：利用與 OpenAI 兼容的端點
- **模型選擇**：根據不同使用場景選擇合適的模型
- **速率限制**：優雅地處理 API 限制
- **錯誤恢復**：穩健的錯誤處理與重試模式

## 🔧 技術架構

### 核心組件
- **Microsoft Agent Framework**：支持 GitHub 模型的 Python 實現
- **GitHub Models API**：訪問最先進的語言模型
- **OpenAI 客戶端模式**：標準化的 API 交互模式
- **環境配置**：安全且靈活的配置管理

### 設計模式的優勢
- **可維護性**：清晰的代碼組織與結構
- **可擴展性**：隨應用需求增長的模式
- **可靠性**：處理邊界情況的成熟方法
- **性能**：高效的資源利用與 API 使用

## ⚙️ 前置條件與設置

**所需依賴項：**
```bash

pip install agent-framework-core  -U

```

**環境配置 (.env 文件)：**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

**GitHub 模型訪問：**
- 擁有模型訪問權限的 GitHub 帳戶
- 具有適當權限的個人訪問令牌
- 了解速率限制與使用模式

## 📚 設計模式分類

### 1. **創建型模式**
- 代理工廠與建造者模式
- 配置管理模式
- 代理服務的依賴注入

### 2. **行為型模式**
- 工具執行與編排
- 對話流程管理  
- 回應處理與格式化

### 3. **整合型模式**
- GitHub Models API 整合
- 錯誤處理與重試邏輯
- 資源管理與清理

## 🚀 展示的最佳實踐

- **乾淨架構**：分層設計，職責明確
- **錯誤處理**：全面的異常管理
- **配置**：基於環境的設置，適應不同環境
- **測試**：支持有效單元測試與整合測試的模式
- **文檔**：自我描述的代碼，意圖清晰

準備好探索專業的代理設計模式了嗎？讓我們一起構建穩健的系統吧！🌟


In [1]:
! pip install agent-framework-core  -U

In [2]:
# 📦 Import Core Libraries for Agent Design Patterns
import os                     # Environment variable access for configuration management
from random import randint    # Random selection utilities for tool functionality

from dotenv import load_dotenv  # Secure environment configuration loading

In [3]:
# 🤖 Import Microsoft Agent Framework Components  
# ChatAgent: Core agent orchestration class following factory pattern
# OpenAIChatClient: GitHub Models integration following adapter pattern
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [4]:
# 🔧 Configuration Loading Pattern
# Implement configuration management pattern for secure credential handling
# This follows the external configuration principle for cloud-native applications
load_dotenv()

True

In [5]:
# 🛠️ Tool Function Design Pattern
# Implements the Strategy Pattern for pluggable agent capabilities
# This demonstrates clean separation of business logic from agent orchestration
def get_random_destination() -> str:
    """Get a random vacation destination using Repository Pattern.
    
    This function exemplifies several design patterns:
    - Strategy Pattern: Interchangeable algorithm for destination selection
    - Repository Pattern: Encapsulates data access logic
    - Factory Method: Creates destination objects on demand
    
    Returns:
        str: A randomly selected destination following consistent format
    """
    # Data Repository Pattern: Centralized destination data management
    destinations = [
        "Barcelona, Spain",      # Mediterranean cultural hub
        "Paris, France",         # European artistic center
        "Berlin, Germany",       # Historical European capital
        "Tokyo, Japan",          # Asian technology metropolis
        "Sydney, Australia",     # Oceanic coastal city
        "New York, USA",         # American urban center
        "Cairo, Egypt",          # African historical capital
        "Cape Town, South Africa", # African scenic destination
        "Rio de Janeiro, Brazil",  # South American beach city
        "Bali, Indonesia"          # Southeast Asian island paradise
    ]
    
    # Factory Method Pattern: Create destination selection on demand
    return destinations[randint(0, len(destinations) - 1)]

In [6]:
openai_chat_client = OpenAIChatClient(base_url=os.environ.get("GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id=os.environ.get("GITHUB_MODEL_ID"))

In [7]:
AGENT_NAME ="TravelAgent"

AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

In [8]:
agent = ChatAgent(
        name = AGENT_NAME,
        chat_client=openai_chat_client,
        instructions=AGENT_INSTRUCTIONS,
        tools=[get_random_destination]
)

In [9]:
thread = agent.get_new_thread()

In [10]:
response1 = await agent.run("Plan me a day trip",thread= thread)

In [11]:

last_message = response1.messages[-1]
text_content = last_message.contents[0].text
print("Travel plan:")
print(text_content)

Travel plan:
Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today? Could you please specify a destination for your day trip?


In [12]:
response2 = await agent.run("I don't like that destination. Plan me another vacation.",thread= thread)

In [13]:
last_message = response2.messages[-1]
text_content = last_message.contents[0].text
print("Change plan:")
print(text_content)

Change plan:
Could you please specify a location or type of vacation you're interested in? This will help me find the perfect place for you!



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
